In [2]:
import pandas as pd
import numpy as np

In [38]:
benign_original = pd.read_csv('VT Dataset/DataSet_Original_Repackaged.csv')  # Original benign apps (VirusTotal scan: 0 detected)
malicious_repackaged = pd.read_csv('VT Dataset/Dataset_Repackaged_Malware.csv')  # Repackaged malware (VirusTotal scan: >=2 detected)
benign_repackage = pd.read_csv('VT Dataset/DataSet_Benign_Repackaged.csv')  # Repackaged benign apps (VirusTotal scan: 0 detected)

COVID_malware = pd.read_csv('VT Dataset/DataSet_COVID_Repackaged_Malware.csv')
COVID_benign = pd.read_csv('VT Dataset/DataSet_COVID_Benign.csv')
other_COVID = pd.read_csv('VT Dataset/Dataset_COVID_Other_Malware.csv')

sensor_usage = pd.read_csv('Sensor Dataset/New_Sensor_Usage.csv').drop(['Unnamed: 0'], axis = 1)
COVID_malware_sensor_usage = pd.read_csv('Sensor Dataset/COVID_Malware_Sensor_Usage.csv').drop(['Category'], axis = 1)
COVID_benign_sensor_usage = pd.read_csv('Sensor Dataset/COVID_Benign_Sensor_Usage.csv').drop(['Category'], axis = 1)
other_COVID_sensor_usage = pd.read_csv('Sensor Dataset/Other_COVID_Malware_Sensor_Usage.csv').drop(['Category'], axis = 1)

In [48]:
original_libs_hash = pd.read_csv('Original_Benign_Libs_Hashes.txt', header = None)
sha256_repack_libs_misuse = pd.read_csv('SHA256_Repackaged_Libs_Misuse.txt', header = None)

sha256_repack_libs_misuse.columns = ['SHA256']
sha256_repack_libs_misuse['if_the_app_using_suspicious_libs'] = 1

In [39]:
COVID_malware_sensor_usage['SHA256'] = [apk.upper() for apk in COVID_malware_sensor_usage['SHA256'].to_list()]
COVID_benign_sensor_usage['SHA256'] = [apk.upper() for apk in COVID_benign_sensor_usage['SHA256'].to_list()]
other_COVID_sensor_usage['SHA256'] = [apk.upper() for apk in other_COVID_sensor_usage['SHA256'].to_list()]

benign_original.rename(columns = {'MD5': 'SHA256'}, inplace = True)

In [40]:
training_dataset = pd.concat([benign_original, malicious_repackaged])
COVID_testset = pd.concat([COVID_malware, COVID_benign, other_COVID])
COVID_sensor_usage = pd.concat([COVID_malware_sensor_usage, COVID_benign_sensor_usage, other_COVID_sensor_usage])

In [41]:
training_dataset = training_dataset[(training_dataset['Malware/Benign'] == 0) | (training_dataset['Malware/Benign'] >= 5)]
COVID_testset = COVID_testset[(COVID_testset['Malware/Benign'] == 0) | (COVID_testset['Malware/Benign'] >= 5)]


In [42]:
training_dataset['label'] = training_dataset['Malware/Benign'].apply(lambda x: 0 if x == 0 else 1)
COVID_testset['label'] = COVID_testset['Malware/Benign'].apply(lambda x: 0 if x == 0 else 1)
benign_repackage['label'] = 0

In [43]:
training_dataset = training_dataset.merge(sensor_usage, how = 'left', on = 'SHA256').drop_duplicates()
test_dataset = benign_repackage.merge(sensor_usage, how = 'left', on = 'SHA256').drop_duplicates()
COVID_testset = COVID_testset.merge(COVID_sensor_usage, how = 'left', on = 'SHA256').drop_duplicates()

In [44]:
labels_training = training_dataset[['label']]
labels_test = test_dataset[['label']]
labels_COVID_test = COVID_testset[['label']]

training_dataset.drop(['label'], axis = 1, inplace = True)
test_dataset.drop(['label'], axis = 1, inplace = True)
COVID_testset.drop(['label'], axis = 1, inplace = True)

In [49]:
training_dataset = training_dataset.merge(sha256_repack_libs_misuse, on = 'SHA256', how = 'left')
test_dataset = test_dataset.merge(sha256_repack_libs_misuse, on = 'SHA256', how = 'left')
training_dataset['if_the_app_using_suspicious_libs'] = training_dataset['if_the_app_using_suspicious_libs'].fillna(0)
test_dataset['if_the_app_using_suspicious_libs'] = test_dataset['if_the_app_using_suspicious_libs'].fillna(0)

In [50]:
training_dataset.drop_duplicates(inplace = True)
test_dataset.drop_duplicates(inplace = True)
COVID_testset.drop_duplicates(inplace = True)

In [63]:
training_dataset.to_csv('Training Dataset/Training_Dataset.csv', index = False)
test_dataset.to_csv('Test Dataset/Repackaged_Benign_Testset.csv', index = False)
COVID_testset.to_csv('Test Dataset/COVID_Testset.csv', index = False)

In [64]:
labels_training.to_csv('Training Dataset/Labels_trainingset.csv', index = False)
labels_test.to_csv('Test Dataset/Labels_testset.csv', index = False)
labels_COVID_test.to_csv('Test Dataset/Labels_COVID_testset.csv', index = False)